# Quantitative Support
This notebooked aims to provide a reproducable way to compute the quantitative statements made in the manuscript. 

We start by loading all relevant data:

In [1]:
import xarray as xr
import datatree as dt
import arviz as az
import pymc as pm
from utils.transformation import yeojohnson_inv


YEAR = 2001
SEASONS = ["DJF", "MAM", "JJA", "SON"]
QUANTITY = "absolute"
VARIABLE = "Plastic"

base_path = f"data/gpr/{QUANTITY}/{VARIABLE}/{YEAR}/"


# In-situ Beach Litter (OSPAR)
ospar = dt.open_datatree("data/beach_litter/ospar/preprocessed.zarr", engine="zarr")
litter_o = ospar["/".join(["preprocessed/", QUANTITY, VARIABLE])]
litter_o = litter_o.sel(year=slice(YEAR, 2020)).dropna("beach_id", **{"how": "all"})


# Effect size, confidence etc.
model_analysis = xr.open_dataset(base_path + "effect_size_seasons.nc")

# Inference data for GP models
idata = {}
for s in SEASONS:
    idata[s] = az.from_netcdf(base_path + f"idata_{s}.nc")


# Posterior predictive of macroplastic
model = dt.open_datatree(base_path + "posterior_predictive.zarr", engine="zarr")
posterior_litter = model["posterior_predictive"][VARIABLE]
# Lambda parameter for Yeo-Johnson transform
lmbda = model["lambda_yeojohnson"]["lambda"].item()

## Supplementary Table 1

In [59]:
for season in SEASONS:
    post = idata[season]["posterior"]
    post_trans = yeojohnson_inv(post["mu_mu"], lmbda)
    lower, higher = pm.hdi(post_trans, hdi_prob=0.95).round(0)["mu_mu"]
    q025, q050, q975 = post_trans.quantile([.025, 0.5, 0.975]).round(0)
    print("{:}: {:} ({:}; {:})".format(season, q050.item(), lower.item(), higher.item()))

DJF: 184.0 (53.0; 352.0)
MAM: 232.0 (79.0; 440.0)
JJA: 154.0 (56.0; 290.0)
SON: 174.0 (73.0; 332.0)


In [62]:
check_vars = ["eta_1", "eta_2", "rho_1", "rho_2", "phi"]
# print(idata["DJF"].posterior.data_vars)
for season in SEASONS:
    print(season)
    post = idata[season]["posterior"]
    hdi = pm.hdi(post, hdi_prob=0.95).round(2)
    vars_q050 = post.quantile(0.5).round(2)
    for v in check_vars:
        print(f"{v}: ", end="")
        lower = hdi[v].sel(hdi="lower").item()
        higher = hdi[v].sel(hdi="higher").item()        
        q050 = vars_q050[v].item()
        print("{:} ({:}; {:})".format(q050, lower, higher))
    print("\n")

DJF
eta_1: 1.08 (0.92; 1.23)
eta_2: 0.64 (0.33; 0.93)
rho_1: 15.75 (10.39; 22.68)
rho_2: 777.24 (235.97; 1965.01)
phi: 1.51 (1.37; 1.66)


MAM
eta_1: 1.19 (1.02; 1.38)
eta_2: 0.72 (0.42; 1.04)
rho_1: 15.65 (10.03; 22.38)
rho_2: 507.78 (214.84; 1096.64)
phi: 1.36 (1.25; 1.48)


JJA
eta_1: 1.08 (0.94; 1.24)
eta_2: 0.65 (0.36; 0.97)
rho_1: 11.92 (7.18; 17.06)
rho_2: 593.49 (205.86; 1451.73)
phi: 1.54 (1.41; 1.68)


SON
eta_1: 1.06 (0.93; 1.21)
eta_2: 0.6 (0.33; 0.9)
rho_1: 8.99 (5.68; 12.91)
rho_2: 690.09 (227.93; 1933.8)
phi: 1.53 (1.41; 1.66)




## Beaches with significant seasonal variations

In [63]:
n_beaches = model_analysis.beach_id.size

confidence_seaosonal_difference = model_analysis.max_hdi.max("combination")

sig_at_95 = (confidence_seaosonal_difference > 0.95).sum().item()
sig_at_80 = (confidence_seaosonal_difference > 0.80).sum().item()

print(f"Significant at 95%: {sig_at_95}")
print(f"Significant at 80%: {sig_at_80}")

print(f"Fraction of significant differences (95%): {sig_at_95 / n_beaches:.3f}")
print(f"Fraction of significant differences (80%): {sig_at_80 / n_beaches:.3f}")


Significant at 95%: 25
Significant at 80%: 66
Fraction of significant differences (95%): 0.149
Fraction of significant differences (80%): 0.393


In [69]:
sig_at_95 = (model_analysis.pvals_cr_mann_whitney.min("combination") < 0.05).sum().item()
sig_at_80 =( model_analysis.pvals_cr_mann_whitney.min("combination") < 0.2).sum().item()

print(f"Fraction of significant differences (95%): {sig_at_95 / n_beaches:.3f}")
print(f"Fraction of significant differences (80%): {sig_at_80 / n_beaches:.3f}")


Fraction of significant differences (95%): 0.119
Fraction of significant differences (80%): 0.238


## What share of beaches are spatially independent?

We assume $3\rho$ (short length scale) to be the distance beyond which spatial autocorrelation doesn't not play an important role anymore. First, we have to compute the pairwise distance matrix for all beaches.

In [93]:
from sklearn.metrics.pairwise import haversine_distances
import numpy as np  

lats = litter_o.lat.values
lons = litter_o.lon.values

rads_y = np.radians(lats)
rads_x = np.radians(lons)

distance_matrix = haversine_distances(list(zip(rads_y, rads_x))) * 6371

rho_autumn = idata["SON"]["posterior"]["rho_1"].median().item()

print("Fraction of beaches which can be considered spatially independent in our dataset")
for season in SEASONS:
    rho = idata[season]["posterior"]["rho_1"].median().item()
    is_within_influence = distance_matrix < 3*rho
    n_beaches_within_influence = is_within_influence.sum(0) - 1  # Minus 1 to exclude the beach itself
    fraction_within_influence = (n_beaches_within_influence == 0).mean()
    print(f"{season}: {fraction_within_influence:.3f}")


Fraction of beaches which can be considered spatially independent in our dataset
DJF: 0.232
MAM: 0.232
JJA: 0.327
SON: 0.399


## How much data of OSPAR was available to Schulz et al. 2013

In [4]:
litter_o.sel(year=slice(2001,2011)).notnull().sum() / litter_o.notnull().sum()

<xarray.DataArray 'Plastic' ()>
array(0.32152095)